In [1]:
import pandas  as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
train_file_path = "/hdd1/sams/avazu/train"

In [3]:
chunk_size = 1000000 # 1,000,000
cnt = 0
col2value = {}
for chunk in tqdm(pd.read_csv(train_file_path, chunksize=chunk_size, index_col='id', dtype=str)):
    # Process the chunk if needed (e.g., data cleaning, manipulation)
    for col in chunk.columns:
        chunk[col].unique()
        if col not in col2value:
            col2value[col] = set()
        unique_values = set(chunk[col].unique())
        col2value[col] = col2value[col] | unique_values

0it [00:00, ?it/s]

41it [10:06, 14.79s/it]


In [4]:
columns_list = chunk.columns

In [5]:
for col in columns_list:
    n = len(col2value[col])
    print(f"column {col} has {n} unique key")

column click has 2 unique key
column hour has 240 unique key
column C1 has 7 unique key
column banner_pos has 7 unique key
column site_id has 4737 unique key
column site_domain has 7745 unique key
column site_category has 26 unique key
column app_id has 8552 unique key
column app_domain has 559 unique key
column app_category has 36 unique key
column device_id has 2686408 unique key
column device_ip has 6729486 unique key
column device_model has 8251 unique key
column device_type has 5 unique key
column device_conn_type has 4 unique key
column C14 has 2626 unique key
column C15 has 8 unique key
column C16 has 9 unique key
column C17 has 435 unique key
column C18 has 4 unique key
column C19 has 68 unique key
column C20 has 172 unique key
column C21 has 60 unique key


In [6]:
# don't need id,
tokenizer = {}
# col_name: col_tokenizer ->
# value : token_id
cnt = 0
for col, values in col2value.items():
    if col in ['click']:
        continue
    tokenizer.setdefault(col, {})
    for val in values:
        val = val
        tokenizer[col][val] = cnt
        cnt += 1

In [7]:
import json
file_path = "/hdd1/sams/avazu/origin_tokenizer.json"
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(tokenizer, f, indent=4)

In [9]:
array_list = []
chunk_size = 1000000 # 1,000,000
# --> 50,000,000
for chunk in tqdm(pd.read_csv(train_file_path, chunksize=chunk_size, index_col='id', dtype=str)):
    # Process the chunk if needed (e.g., data cleaning, manipulation)
    chunk['click'] = chunk['click'].astype(int)
    for col, col_tokenizer in tokenizer.items():
        chunk[col] = chunk[col].map(col_tokenizer)
    array_list.append(chunk.values)

41it [22:19, 32.66s/it]


In [10]:
feature = np.concatenate(array_list, axis=0)
file_path = "/hdd1/sams/avazu/data.npy"
np.save(file_path, feature)

In [15]:
train_ratio = 0.8
test_ratio = 0.1
val_ratio = 0.1
data = feature

num_samples = data.shape[0]
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)
num_test = num_samples - num_train - num_val

# Shuffle the data
np.random.shuffle(data)

# Split the data
train_data = data[:num_train]
val_data = data[num_train:num_train + num_val]
test_data = data[num_train + num_val:]

In [16]:
path = "/hdd1/sams/data/avazu"
np.save(path + '/train.npy', train_data)
np.save(path + '/val.npy', val_data)
np.save(path + '/test.npy', test_data)

In [17]:
train_data.max()

6729485

In [18]:
test_data.max()

6729482

In [19]:
val_data.max()

6729485

In [20]:
feature.shape[1]

23